Step 2 - MongoDB and Flask Application
Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

Start by converting your Jupyter notebook into a Python script called scrape_mars.py with a function called scrape that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

Next, create a route called /scrape that will import your scrape_mars.py script and call your scrape function.

Store the return value in Mongo as a Python dictionary.
Create a root route / that will query your Mongo database and pass the mars data into an HTML template to display the data.

Create a template HTML file called index.html that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

In [1]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd

import time
from selenium import webdriver
from datetime import datetime

In [ ]:
def init_browser():
    # Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [ ]:
def scrape():
    # Initialize browser
    browser = init_browser()
    mars_data = {}
   
    NASANewsurl = 'https://mars.nasa.gov/news/'
    browser.visit(NASANewsurl)

    # Scrape page into soup
    html = browser.html
    soup = bs(html, 'html.parser')
    
     # Find news text and paragraph
    news_title = soup.find('div', class_="content_title").text
    news_p = soup.find('div', class_="article_teaser_body").text

    mars_data['news_title'] = news_title
    mars_data['news_p'] = news_p
    
    image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(image_url)
    html=browser.html
    mars_image = bs(html, 'html.parser')
    full_image = mars_image.find("a",{'id': 'full_image', 'data-fancybox-href': True}).get('data-fancybox-href')
    image_url = f'https://www.jpl.nasa.gov{full_image}'
    mars_data['featured_image'] = image_url
    
    twitter_url = 'https://twitter.com/marswxreport?lang=en&lang=en'
    browser.visit(twitter_url)
    html = browser.html
    mars_weather = bs(html, 'html.parser')
    mars_tweet = mars_weather.find('div', class_='js-tweet-text-container').get_text()
    mars_data['weather'] = mars_tweet
    
    facts_url = 'https://space-facts.com/mars/'
    mars_facts = pd.read_html(facts_url)
    df = pd.DataFrame(mars_facts[0])
    df.columns = ['Mars', 'Info']
    mars_table = df.set_index('Mars')
    html_table = mars_table.to_html()
    html_table.replace('\n', '')
    mars_data['facts'] = html_table

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemipshere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]
```

In [4]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [5]:
html = browser.html

In [6]:
mars_hemispheres= bs(html, 'html.parser')

In [9]:
hemispheres = mars_hemispheres.find_all("div", class_="description")
hemispheres

#'div', class_='description'
#<a href="/search/map/Mars/Viking/cerberus_enhanced" class="itemLink product-item"><h3>Cerberus Hemisphere Enhanced</h3></a>

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [13]:
hemisphere_image_urls = []
url_ = 'https://astrogeology.usgs.gov'

for hemisphere in hemispheres:
   
    img_title = hemisphere.h3.text
    hemisphere_url = url_ + hemisphere.a['href']

   
    browser.visit(hemisphere_url) 
    img_html = browser.html
    
    soup = bs(img_html, 'html.parser')
    img_url = soup.find('div', class_='downloads').find('li').a['href']

    #Save image and title in a dictionary 
    
    mars_dict = {'title': img_title, 'img_url': hemisphere_url}

    #append dictionary
    hemisphere_image_urls.append(mars_dict)
    print(hemisphere_image_urls)








[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'}]
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'}]
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'}]
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeolo